# Natural Language Processing (NLP)

- NLP is a very large field of machine learning that focuses on creating models from a text data source 
- This course is a rapid presentation of tools provided by Spark machine learning library that facilitate constructing these models.  
- Applications of NLP are numerous:

    - Books recommendations
    - News articles clustering
    - Sentimental analysis
    - Spam detection
    - etc. 

## Terminology

### Document
- A document can be an email, a text feedback, an sms, an article, a book, etc.. It is represented as a vector of word counts is called a “Bag of Words

### Corpus
- corpus (plural, "corpora") is a set of documents

## Process
- Compile the corpus and create a DataFrame
- Featurization: it is about preparing the features in order to create the model. This may includes:

    - Tokenization. Spark tools in pyspark.ml.feature: Tokenizer, RegexTokenizer
    - Stop Words Removing. Spark tools in pyspark.ml.feature: StopWordsRemover
    - Using NGrams. Spark tools in pyspark.ml.feature: NGram
    - Numeric Features. Spark tools in pyspark.ml.feature include HashingTF, IDF, CountVectorizer

In [ ]:
import findspark

In [ ]:
findspark.init("/home/nahle/spark-2.1.0-bin-hadoop2.7")

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('NLP_starter').getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [ ]:
df= spark.createDataFrame([
    (0, 'Hi Spark is good'),
    (1, 'Java is good choice for beginners'),
    (2, 'In,Spark,you can, do, logistic,Regression;NLP'),
    (3, 'Thanks for using spark. Many corporate will appreciate that!'),
    (4, 'Learn about re on  https://docs.python.org/3/library/re.html')
], ['label', 'text']
)

In [ ]:
df.show(truncate=False)

# Start with Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')

In [ ]:
tokenized_df = tokenizer.transform(df)

In [ ]:
tokenized_df.show(truncate=False)

In [ ]:
tokenized_output = tokenized_df.collect()

In [ ]:
for token in tokenized_output[4]:
    print (token)

In [ ]:
tokenized_df.head()

In [ ]:
tokenized_df.take(1)

In [ ]:
from pyspark.sql.functions import udf 
from pyspark.sql.types import IntegerType
count_words = udf(lambda words: len(words), IntegerType())

In [ ]:
tokenized_df.withColumn('counts', count_words('words')).show()

# Now let's use RegexTokenizer

In [ ]:
regex_tokenizer = RegexTokenizer(inputCol='text', outputCol='words', pattern='\\W')
#regex_tokenizer.setMinTokenLength(4)

## More on regular expressions with Python
More on:
https://docs.python.org/3/library/re.html

### \W
Matches any character which is not a word character. This is the opposite of \w. If the ASCII flag is used this becomes the equivalent of [^a-zA-Z0-9_] (but the flag affects the entire regular expression, so in such cases using an explicit [^a-zA-Z0-9_] may be a better choice). 

In [ ]:
regex_df = regex_tokenizer.transform(df)

In [ ]:
regex_tokenized_counts = regex_df.withColumn('freq', count_words('words'))

In [ ]:
regex_tokenized_counts.show()

In [ ]:
for item in regex_tokenized_counts.take(5)[4]:
    print (item)

# With RegexTokenizer, we can remove small words 

In [ ]:
regex_tokenizer_min_4 = RegexTokenizer(inputCol='text', outputCol='words', pattern='\\W')
regex_tokenizer_min_4.setMinTokenLength(4)
regex_min_4_df = regex_tokenizer_min_4.transform(df)
regex_tokenized_min_4_counts = regex_min_4_df.withColumn('freq', count_words('words'))
regex_tokenized_min_4_counts.show()

In [ ]:
for item in regex_tokenized_min_4_counts.take(5)[4]:
    print (item)

# Once we have the tokens, we can remove _stop-words_ by using _StopWordsRemover_

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
remover = StopWordsRemover(inputCol='words', outputCol='tokens')

In [ ]:
tokens_filtered = remover.transform(regex_tokenized_counts)

In [ ]:
cleanDF= tokens_filtered.withColumn('count_tokens', count_words('tokens'))


In [ ]:
cleanDF.select('words', 'freq', 'tokens', 'count_tokens').show()

In [ ]:
cleanDF.show()

In [ ]:
for item in cleanDF.collect()[4]:
   print(item)    

In [ ]:
remover.getStopWords()

In [ ]:
stopWords=['a', 'is', 'for', 'hi', 'in', 'on']
remover.setStopWords(stopWords)


In [ ]:
remover.transform(regex_tokenized_counts).select('text', 'tokens').show(truncate=False)

In [ ]:
newCleanDF=remover.transform(regex_tokenized_counts).withColumn('count_tokens', count_words('tokens'))
newCleanDF.show()


In [ ]:
for item in newCleanDF.collect()[4]:
   print(item)  

# NGRAM
A feature transformer that converts the input array of strings into an array of n-grams

In [ ]:
from pyspark.ml.feature import NGram

In [ ]:
cleanDF.show()

In [ ]:
ngram = NGram(n=2, inputCol='tokens', outputCol='2grams')

In [ ]:
my_2ngrams =ngram.transform(cleanDF)

In [ ]:
my_2ngrams.show()

In [ ]:
my_2ngrams.select('2grams').show(truncate =False)

In [ ]:
for item in my_2ngrams.collect()[4]:
    print (item)

# TF: Term Frequency
Maps a sequence of terms to their term frequencies using the hashing trick.

Note: the terms must be hashable (can not be dictionary or list...).

HashingTF(S) takes the hash code of each word modulo the desired vector size S, and thus maps each word to a number between 0 and S-1.

This yields a quite robust vector even if multiple words may map to the same hash code. 

Spark Machine Learning developers recommend setting S between $2^{18}$ and $2^{20}$


In [ ]:
from pyspark.ml.feature import HashingTF, IDF

In [ ]:
cleanDF.show()

In [ ]:
tf = HashingTF(numFeatures=1000, inputCol='tokens', outputCol='features')

In [ ]:
tf.explainParams()


In [ ]:
tf.explainParam('numFeatures')

In [ ]:
tf.setNumFeatures(256)

In [ ]:
print (tf.getNumFeatures())

In [ ]:
tf_df = tf.transform(cleanDF)

In [ ]:
tf_df.show()

In [ ]:
for item in tf_df.collect()[4]:
    print (item)

# TF-IDF: Term Frequence - Inverse Document Frequency

Once you have TF vectors, you can use IDF to compute the inverse document frequencies and multiply them with the TF to compute the TF-IDF 

IDF measures how infrequently a term occurs across the whole document corpus

TF x IDF shows how relevant a term is to specific document (i.e., if it is common in that document but rare in the whole corpus)

TF-IDF is used to improve on Bag of Words by adjusting word counts based on their frequency in the corpus

## How to calculate them?
Various ways for determining the exact values of both statistics exist:

- TF(x, y): number of occurences of term x in document y. It represents the importance of a term in the document. 

- IDF(t): Importance of the term in the document. 

$$IDF(t)= log\frac{N}{N(t)}$$
- N: number of documents in the corpus D, N=|D|
- N(t): Number of the documents where the term t appears (i.e: TF(t, d)!=0). N(t)= |{d in D, t in D}|
- TF-IDF(t, d) =TF(t, d)  

## Further Information on TF-IDF
https://fr.wikipedia.org/wiki/TF-IDF

## Preparing Data to TF-IDF
In a real pipeline, you will likely need to preprocess and stem words before passing them to TF.

Ex: convert words to lowercase, drop punctuation characters or drop suffixes like ‘ing’.

You can use external single node natural language libraries like NLTK (http://www.nltk.org) 

In [ ]:
idf = IDF(inputCol='features', outputCol='idf_features')

In [ ]:

idf_model = idf.fit(tf_df)

In [ ]:
data=idf_model.transform(tf_df)

In [ ]:
data.show()

In [ ]:
for term in data.collect()[4]:
    print (term)

In [ ]:
data.select('tokens').show(truncate = False)

# CountVectorizer

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
count_vec = CountVectorizer(inputCol='tokens', outputCol='features',  minDF=1)

In [ ]:
model = count_vec.fit(cleanDF)

In [ ]:
data = model.transform(cleanDF)

In [ ]:
data.select(['tokens', 'features']).show(truncate = False)

In [ ]:
count_vec = CountVectorizer(inputCol='tokens', outputCol='features', minDF=2)

In [ ]:
help(CountVectorizer)

In [ ]:
count_vec.explainParam('minDF')

In [ ]:
count_vec_df=count_vec.fit(cleanDF).transform(cleanDF).select('tokens', 'features')

In [ ]:
count_vec_df.show(truncate = False)

In [ ]:
count_vec_df.printSchema()

In [ ]:
count_vec = CountVectorizer(inputCol='tokens', outputCol='features', vocabSize=15, minDF=1)

In [ ]:
count_vec.fit(cleanDF).transform(cleanDF).select('tokens', 'features').show(truncate = False)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator